In [1]:
# Run this block to install dependencies [Remember to make the statement true]
if 1 == 1:
    !pip install pandas
    !pip install tqdm
    !pip install scikit-learn
    !pip install gensim
    !pip install spacy
    !python -m spacy download en
    !pip install numpy

    100% |████████████████████████████████| 37.4MB 58.6MB/s ta 0:00:01

    Linking successful
    /home/la/anaconda3/envs/py35/lib/python3.5/site-packages/en_core_web_sm
    -->
    /home/la/anaconda3/envs/py35/lib/python3.5/site-packages/spacy/data/en

    You can now load the model via spacy.load('en')



In [1]:
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')

# Import W2V library
from gensim.models.word2vec import Word2Vec

import pandas as pd
import spacy
import pickle
from tqdm import tqdm
from gensim.corpora import Dictionary
from sklearn.utils import shuffle
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

nlp = spacy.load('en')
DATA_LIMIT = 1000

df = pd.read_csv('./imdb_master.csv', encoding='latin1')
df_neg = df[df['label'] == 'neg']
df_pos = df[df['label'] == 'pos']
df = pd.concat((df_pos[:DATA_LIMIT], df_neg[:DATA_LIMIT]))

def process_text(input_string, return_string=False, stem=False):
    text = nlp(u'' + input_string)
    if stem == True:
        text = [tok.lemma_ for tok in text if (tok.is_alpha and not tok.is_stop)]
    else:
        text = [tok.lower_ for tok in text if (tok.is_alpha and not tok.is_stop)]
    if return_string == True:
        return " ".join(text)
    return text

In [2]:
df.head()

,Unnamed: 0,type,review,label,file
12500,12500,test,I went and saw this movie last night after bei...,pos,0_10.txt
12501,12501,test,Actor turned director Bill Paxton follows up h...,pos,10000_7.txt
12502,12502,test,As a recreational golfer with some knowledge o...,pos,10001_9.txt
12503,12503,test,"I saw this film in a sneak preview, and it is ...",pos,10002_8.txt
12504,12504,test,Bill Paxton has taken the true story of the 19...,pos,10003_8.txt


In [3]:
len(df)

2000

In [4]:
tqdm??

In [5]:
nlp??

In [6]:
df.shape

(2000, 5)

In [7]:
tokens = nlp('An example sentence. Another example sentence.')
tokens[0].text, tokens[0].head.tag_

('An', 'NN')

In [8]:
len(tokens)

8

In [9]:
tokens[3].text

'.'

In [10]:
# Make this statement true to run from scratch [It takes time to process the text]
if 1 == 1:
    wordlist = []
    for i in tqdm(range(df.shape[0])):
        wordlist.append(process_text(df['review'].iloc[i]))
        
    with open('vocabulary.txt', 'wb') as vocabulary:
        pickle.dump(wordlist, vocabulary)
    vocabulary.close()

100%|██████████████████████████████████████████████████████████████████████████████| 2000/2000 [04:16<00:00,  7.80it/s]


In [11]:
import multiprocessing
# hs = Hierarchical Softmax, min_count -> excludes token less than that frequency, 
w2v_model = Word2Vec(window=5, workers=multiprocessing.cpu_count(), size=300, iter=100, min_count=1, hs=1, negative=0)

In [12]:
print(w2v_model)

Word2Vec(vocab=0, size=300, alpha=0.025)


In [13]:
w2v_model.build_vocab(wordlist)

In [14]:
print(w2v_model)

Word2Vec(vocab=22449, size=300, alpha=0.025)


In [15]:
%time w2v_model.train(wordlist, total_examples=w2v_model.corpus_count, epochs=w2v_model.epochs)

Wall time: 1min 10s


(20548792, 21977100)

In [16]:
w2v_model.wv['cat'].shape

(300,)

In [17]:
vocabulary = Dictionary(wordlist)

In [18]:
len(vocabulary)

22449

In [19]:
w2v_model.wv.most_similar('bellucci')

[('belucci', 0.3595184087753296),
 ('romane', 0.3292483687400818),
 ('inflatable', 0.3289024829864502),
 ('dolan', 0.28767144680023193),
 ('badger', 0.27900922298431396),
 ('inverting', 0.2763495445251465),
 ('limos', 0.2744201719760895),
 ('enhancement', 0.2607278823852539),
 ('thumb', 0.2559332847595215),
 ('muriel', 0.2539374828338623)]

In [21]:
for resultant in w2v_model.wv.most_similar(positive=['movie', 'good'], negative=['bad']):
    print(resultant)

('cinema', 0.24760204553604126)
('wood', 0.2246340662240982)
('ramin', 0.21162863075733185)
('flick', 0.2085748016834259)
('fantsastic', 0.1930416077375412)
('watcha', 0.1919766664505005)
('audience', 0.18605846166610718)
('end', 0.18570637702941895)
('synopsis', 0.18509164452552795)
('alternative', 0.18284578621387482)



1. Create a numpy empty/random matrix with dimension of [Vocsize+1 X 
embedding dimension]
2. Load the embeddings into that word
3. Create keras embedding layer with the same configuration and 
load weights there
4. Train a RNN/CNN to classify [optional]

In [22]:
embedding_dim = 300
embedding = np.zeros((len(vocabulary)+1, embedding_dim))
embedding[0] = 0
embedding[1] = w2v_model.wv[ vocabulary[0] ]

In [23]:
embedding.shape

(22450, 300)

In [24]:
for i in range(len(vocabulary)):
    embedding[i+1] = w2v_model.wv[vocabulary[i]]

In [25]:
embedding_dim = 300
embedding = np.zeros((len(vocabulary)+1, embedding_dim))
for i in range(len(vocabulary)):
    embedding[i + 1] = w2v_model.wv[vocabulary[i]]

In [26]:
!pip install keras

Exception:
Traceback (most recent call last):
  File "c:\program files (x86)\python36-32\lib\site-packages\pip\basecommand.py", line 215, in main
    status = self.run(options, args)
  File "c:\program files (x86)\python36-32\lib\site-packages\pip\commands\install.py", line 342, in run
    prefix=options.prefix_path,
  File "c:\program files (x86)\python36-32\lib\site-packages\pip\req\req_set.py", line 784, in install
    **kwargs
  File "c:\program files (x86)\python36-32\lib\site-packages\pip\req\req_install.py", line 851, in install
    self.move_wheel_files(self.source_dir, root=root, prefix=prefix)
  File "c:\program files (x86)\python36-32\lib\site-packages\pip\req\req_install.py", line 1064, in move_wheel_files
    isolated=self.isolated,
  File "c:\program files (x86)\python36-32\lib\site-packages\pip\wheel.py", line 345, in move_wheel_files
    clobber(source, lib_dir, True)
  File "c:\program files (x86)\python36-32\lib\site-packages\pip\wheel.py", line 316, in clobber
    en

In [158]:
!pip install tensorflow

    100% |████████████████████████████████| 45.9MB 34kB/s  eta 0:00:01 6% |██                              | 2.9MB 6.6MB/s eta 0:00:07
  Using cached gast-0.2.0.tar.gz
    100% |████████████████████████████████| 7.5MB 181kB/s eta 0:00:01
  Using cached astor-0.6.2-py2.py3-none-any.whl
  Using cached tensorboard-1.6.0-py3-none-any.whl
    100% |████████████████████████████████| 81kB 5.6MB/s eta 0:00:01
    100% |████████████████████████████████| 6.4MB 234kB/s eta 0:00:01
  Using cached Markdown-2.6.11-py2.py3-none-any.whl
  Using cached html5lib-0.9999999.tar.gz
  Running setup.py bdist_wheel for gast ... done
  Stored in directory: /home/la/.cache/pip/wheels/8e/fa/d6/77dd17d18ea23fd7b860e02623d27c1be451521af40dd4a13e
  Running setup.py bdist_wheel for absl-py ... done
  Stored in directory: /home/la/.cache/pip/wheels/3c/0f/0a/6c94612a8c26070755559045612ca3645fea91c11f2148363e
  Running setup.py bdist_wheel for html5lib ... done
  Stored in directory: /home/la/.cache/pip/wheels/6f/85/6c

In [27]:
import keras

Using TensorFlow backend.


In [28]:
from keras.models import Sequential
from keras.layers import Embedding, GRU, Dense

In [161]:
Embedding??

In [29]:
embd = Embedding(len(vocabulary)+1,output_dim=embedding_dim, weights = [embedding], trainable =False, input_length=100)

In [30]:
print(embd.output_dim)

300


# Preprocessing

In [31]:
X = []
for review in wordlist:
    X.append(np.array(vocabulary.doc2idx(review)) + 1)

In [32]:
from keras.preprocessing.sequence import pad_sequences

In [33]:
train_x = pad_sequences(X, value=0, maxlen=200)

In [34]:
train_y = np.concatenate((np.ones(1000), np.zeros(1000)))

In [35]:
from sklearn.utils import shuffle
train_x, train_y = shuffle(train_x, train_y)
from keras.layers import Bidirectional
model = Sequential()
model.add(Embedding(len(vocabulary) + 1, 300, weights=[embedding], trainable=False))
model.add(GRU(128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
model.add(Bidirectional(GRU(50, dropout=0.2)))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(train_x, train_y,
          batch_size=128,
          epochs=15, validation_split=0.3)

Train...
Train on 1400 samples, validate on 600 samples
Epoch 1/15
1400/1400 [==============================] - ETA: 1:20 - loss: 0.7267 - acc: 0.484 - ETA: 50s - loss: 0.7152 - acc: 0.480 - ETA: 38s - loss: 0.7058 - acc: 0.50 - ETA: 30s - loss: 0.7023 - acc: 0.50 - ETA: 24s - loss: 0.6943 - acc: 0.52 - ETA: 19s - loss: 0.6930 - acc: 0.52 - ETA: 15s - loss: 0.6874 - acc: 0.53 - ETA: 11s - loss: 0.6835 - acc: 0.55 - ETA: 7s - loss: 0.6794 - acc: 0.5608 - ETA: 3s - loss: 0.6765 - acc: 0.569 - 45s 32ms/step - loss: 0.6737 - acc: 0.5721 - val_loss: 0.6227 - val_acc: 0.6650
Epoch 2/15
1400/1400 [==============================] - ETA: 29s - loss: 0.6083 - acc: 0.66 - ETA: 27s - loss: 0.5977 - acc: 0.69 - ETA: 25s - loss: 0.5849 - acc: 0.72 - ETA: 23s - loss: 0.5804 - acc: 0.73 - ETA: 19s - loss: 0.5677 - acc: 0.75 - ETA: 16s - loss: 0.5617 - acc: 0.75 - ETA: 12s - loss: 0.5648 - acc: 0.74 - ETA: 9s - loss: 0.5622 - acc: 0.7500 - ETA: 6s - loss: 0.5582 - acc: 0.750 - ETA: 2s - loss: 0.5545 - 